# Inferencia de Componentes Principales

Autor: Rafaella Ormond y Jose Jaime Martinez-Magana

Este script realiza un Análisis de Componentes Principales (PCA) para generar covariables para el GWAS de rasgos de tabaquismo en las cohortes LAGC.

**ENTRADA:** Archivos binarios de PLINK2 `(.pgen/.psan/.pvar)` + genoma de referencia `(.fa)` → control de calidad y conversión de formato
**SALIDA:** Archivos VCF controlados por calidad, divididos por cromosoma (chr1-22), listos para imputación en los servidores TOPMed/Michigan

### ***Requisitos:***
### Descargar Plink
Podemos descargar Plink versión 1.9 y versión 2.0 siguiendo los pasos de su sitio web.<br>
Para instalar plink2 [acceda aquí](https://www.cog-genomics.org/plink/2.0/)<br>
Para instalar plink1.9 [acceda aquí](https://www.cog-genomics.org/plink/1.9/) <br>

### Descargar e instalar R

Puede descargar e instalar R desde el Comprehensive R Archive Network (CRAN) para su sistema operativo:

- [R para Windows](https://cran.r-project.org/bin/windows/base/)
- [R para macOS](https://cran.r-project.org/bin/macosx/)
- [R para Linux](https://cran.r-project.org/bin/linux/)

Después de instalar R, puede instalar los paquetes requeridos como se muestra a continuación.

### Descargar paquetes de R
Instale los siguientes paquetes de R (haga clic en el nombre para visitar la página web de cada paquete):
- [GENESIS](https://bioconductor.org/packages/release/bioc/html/GENESIS.html)
- [GWASTools](https://bioconductor.org/packages/release/bioc/html/GWASTools.html)
- [SNPRelate](https://bioconductor.org/packages/release/bioc/html/SNPRelate.html)
- [SeqArray](https://bioconductor.org/packages/release/bioc/html/SeqArray.html)
- [parallel](https://stat.ethz.ch/R-manual/R-devel/library/parallel/doc/parallel.pdf) (paquete incorporado en R)
- [BiocParallel](https://bioconductor.org/packages/release/bioc/html/BiocParallel.html)
- [stringi](https://cran.r-project.org/package=stringi)

**Nota:** La mayoría de los paquetes de Bioconductor se pueden instalar usando los siguientes comandos en R:

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install(c("GENESIS", "GWASTools", "SNPRelate", "SeqArray", "BiocParallel"))
install.packages("stringi")

### Pasos del análisis:
1) Poda de LD usando PLINK
2) Creación de archivos GDS<br>
    2.1. Creación del script para crear los archivos gds<br>
    2.2 Crear los archivos gds
3) Cálculo de Componentes Principales usando PC-AiR
   3.1. Desarrollo del script para PCA y GRM con PCAiR y PCRelate<br>
   3.2. Generar los archivos de PCA y GRM<br>
   3.3. Guardar el archivo de PCA

### 1. Pruning de LD usando PLINK

***Descripción:***
Este código es para realizar pruning de LD usando PLINK2 para el análisis de componentes principales.

El formato plink2 utiliza pfiles, también se puede hacer la transformación para bfiles, pero si los archivos están divididos por cromosomas, con pfiles es más rápido unirlos.

In [ ]:
## First, we will perform an LD pruning with plink2
# Substitute for the path and file name prefix of plink2 files
in="/path_to_your_data/plink_prefix"
# Perform first pass of LD pruning
plink2 --pfile ${in}\
    --indep-pairwise 50 5 0.2 --geno 0.01 --mind 0.01 --maf 0.05\
    --out ${in}_indep_snps
plink2 --pfile ${in}\
    --extract ${in}_indep_snps.prune.in --make-bed\
    --out ${in}_forpcair

### 2. Creación de archivos GDS

***Descripción:***

Este script creará los archivos gds que se usarán para realizar el cálculo de Componentes Principales en el siguiente paso.

### 2.1 Creación del script para crear los archivos gds
Nombre del script: **create_gds.Rscript**

Puede descargar este script en github [ENLACE AQUÍ](https://github.com/ormondr/Smoking_GWAS_LAGC/blob/main/English/01PC/create_gds.Rscript)

In [ ]:
####################################################################################
# This script uses a plink file to create a GDS file for input to PCAiR
####################################################################################
# set parameters
# this function uses the library optparse to add arguments to the script
# adding arguments to the script
library(optparse) 
option_list = list(
    make_option(c("--plinkfile"), type="character", default=NULL,
                help="path to the directory storing the plink files, without plink extensions *bed/*bim/*fam", metavar="character"),
    make_option(c("--out"), type="character", default=NULL,
                help="output file name for GDS file, use complete path, example /data/analsyis/analysis.gds", metavar="character")
);

opt_parser = OptionParser(option_list=option_list);
opt = parse_args(opt_parser);

if (is.null(opt$plinkfile)){
    print_help(opt_parser)
    stop("At least one argument must be supplied (input file)", call.=FALSE)
}

###################################################################################
# load libraries
library(GENESIS)
library(GWASTools)
library(SNPRelate)

# create list of files
bed_f=paste0(opt$plinkfile,".bed",sep="")
bim_f=paste0(opt$plinkfile,".bim",sep="")
fam_f=paste0(opt$plinkfile,".fam",sep="")

# create gds output files
snpgdsBED2GDS(bed.fn=bed_f,
              bim.fn=bim_f,
              fam.fn=fam_f,
              family=TRUE,
              out.gdsfn=opt$out)
# end of script

### 2.2 Crear los archivos gds
***Descripción:***
Este script ejecutará el script "create_gds.Rscript" creado en el paso anterior.<br>
Obs: necesita modificar los parámetros para sus archivos.

In [ ]:
# creating GDS files
# set parameters
Rscript='/path_to_your_data/create_gds.Rscript'
inpath='/path_to_your_data/01plinkldpruned'
oupath='/path_to_your_data/02gdsprunned'

# create job list file
do Rscript ${Rscript} --plinkfile=${inpath}/$cohort_name_hapmap_metalprs_allchrs --out=${oupath}/cohort_name_hapmap_metalprs_allchrs.gds

### 3. Cálculo de Componentes Principales usando PC-AiR

**Descripción:**

Este script realiza el Análisis de Componentes Principales (PCA), que se utilizará como covariables en el GWAS.
El PCA se realiza usando la función `PC-AiR` del paquete **GENESIS**.

- Para aprender más sobre PC-AiR y PC-Relate, consulte esta viñeta: [Guía PCA GENESIS](https://bioconductor.org/packages/devel/bioc/vignettes/GENESIS/inst/doc/pcair.html)
  - Ver **Sección 3** para PC-AiR y **Sección 4** para PC-Relate.

> **Nota:**
> El PCA requiere poda de LD previa, que se realiza en el primer paso de este flujo de trabajo.
> El archivo que contiene los 10 principales componentes se usará para todos los métodos GWAS (por ejemplo, Regenie, GMMAT, Saige).
> El archivo GRM solo es necesario para GMMAT, pero recomendamos generarlo de todos modos — así no tendrá que repetir este paso si decide usar GMMAT más adelante.

### 3.1. Desarrollo del script para PCA y GRM con PCAiR y PCRelate
*Script escrito en R.*
Desarrollo del script para PCA y GRM con PCAiR y PCRelate <br>
Este script no necesita ser modificado, las rutas y nombres de archivos se agregarán en el siguiente paso<br>
Nombre del script: **create_grm.Rscript**

Puede descargar este script en github [ENLACE AQUÍ](https://github.com/ormondr/Smoking_GWAS_LAGC/blob/main/English/01PC/create_grm.Rscript)

In [ ]:
####################################################################################
# R script for creating GRM and PCs
# day: 25 July 2025
# author: Rafaella Ormond and Jose Jaime Martinez-Magana
####################################################################################
# This script uses a gds file to estimate PCs and GRM using GENESIS
####################################################################################
# set parameters
# this function uses the library optparse to add arguments to the script
# adding arguments to the script
library(optparse) 
option_list = list(
    make_option(c("--gdsfile"), type="character", default=NULL,
                help="path to gds file", metavar="character"),
    make_option(c("--out"), type="character", default=NULL,
                help="output path for PCAiR objects, use a complete directory path and a potential name of files. Example: /home/user/out_pca", metavar="character")
);

opt_parser = OptionParser(option_list=option_list);
opt = parse_args(opt_parser);

if (is.null(opt$gdsfile)){
  print_help(opt_parser)
  stop("At least one argument must be supplied (input file)", call.=FALSE)
}

###################################################################################
# load libraries
library(GENESIS)
library(GWASTools)
library(SNPRelate)
library(SeqArray)
library(parallel)
library(BiocParallel)
library(stringi);

# stablishing a seed
set.seed(1000)
# stablishing cores
cores=detectCores()

# create function for LD prunning
ld_prun=function(gds){
snpset=snpgdsLDpruning(gds,
                       method="corr",
                       slide.max.bp=10e6,
                       ld.threshold=sqrt(0.1),
                       maf=0.01,
                       missing.rate=0.01,
                       verbose=TRUE, num.thread=cores);
    pruned=unlist(snpset,
                  use.names=FALSE);
return(pruned)
}

# create function for KING-robust analysis
king_mat=function(gds){
samp.id=read.gdsn(index.gdsn(gds, "sample.id"))
ibd.robust=snpgdsIBDKING(gds, sample.id=samp.id, family.id=NULL, maf=0.01,missing.rate=0.01,num.thread=cores)
return(ibd.robust)
}


# create function for PCAiR
pcair_r=function(gds_geno, pruned, KINGmat){
pcair=pcair(gds_geno, snp.include=pruned,
            kinobj=KINGmat, divobj=KINGmat)
return(pcair)
}

# create function for PCAiR with sample filter
pcair_r_fil=function(gds_geno, pruned, KINGmat, sample_list){
    pcair=pcair(gds_geno, snp.include=pruned,
                sample.include=sample_list,
                kinobj=KINGmat, divobj=KINGmat)
                return(pcair)
                }


# create function to match the sample ID in the GDS with the sample IDs of the supplied sampleID filter
filter_samples=function(gds_samples, incl_samples){
    matches=unique(grep(paste(incl_samples$SampleID,collapse="|"),
                        gds_samples, value=TRUE))
    return(matches)
}


# run analysis for pcs with relationships
# open the gds object
gds=snpgdsOpen(opt$gdsfile);
# LD prunning
pruned=ld_prun(gds)
# build KING matrix
KINGmat=king_mat(gds)
# adjust KING matrix
KINGmat_m=KINGmat$kinship
# add sampleIDs to colnames y row names
colnames(KINGmat_m)=KINGmat$sample.id
rownames(KINGmat_m)=KINGmat$sample.id
# get samples in gds
gds_samples=read.gdsn(index.gdsn(gds, "sample.id"))
# close the gds object
snpgdsClose(gds)

# read the gds object
gds=GdsGenotypeReader(filename=opt$gdsfile)

# create a GenotypeData class object
gds_geno=GenotypeData(gds)

# run PCAiR
PCair=pcair_r(gds_geno, pruned, KINGmat_m)
PCairpart=pcairPartition(kinobj = KINGmat_m, divobj = KINGmat_m)

# PC relate calculation 
gdsData=GenotypeBlockIterator(gds_geno, snpInclude=pruned)
PCrelate=pcrelate(gdsData,
                  pcs=PCair$vectors[,1:2],
                  training.set=PCairpart$unrels,
                  BPPARAM=BiocParallel::SerialParam())

# making GRM
grm=pcrelateToMatrix(PCrelate)
# making sparce matrix
grm[grm<0.05]=0
grm_sparse=as.matrix(grm, sparse = TRUE)

# save file
out=list()
out$PCair=PCair
out$PCairpart=PCairpart
out$KINGmat=KINGmat_m
out$pruned=pruned
out$PCrelate=PCrelate
out$grm=grm
out$grm_sparse=grm_sparse
saveRDS(file=paste0(opt$out,".rds",""),out)

### 3.2. Generar los archivos de PCA y GRM.
***Descripción:***

Este script ejecutará el "create_grm.Rscript" creado en el paso anterior.
Deberá ajustar este paso según sus propios archivos de entrada.
La salida será un archivo .RDS que contendrá tanto la información de PCA como la Matriz de Relaciones Genéticas (GRM).

In [ ]:
# run R script for creating GDS files
# add the path of the create_grm.Rscript from the previous step
Rscript="create_grm.Rscript"
Rscript $Rscript \
--gdsfile=gdsfile_prunned \
--out=cohort_name.gds_prunned_grm_pca

### 3.3. Guardar el archivo de PCA

***Descripción:***
Este script extrae los 10 principales componentes del archivo .RDS generado en el paso anterior y los guarda en un archivo separado para ser usados como covariables en Regenie.

In [ ]:
# Load the PCA/GRM result file
pca_grm <- readRDS("cohort_name.gds_prunned_grm_pca.rds")

# Extract sample IDs (IIDs)
iids <- rownames(pca_grm$PCair$vectors)

# Extract top 10 PCs
pcs <- pca_grm$PCair$vectors[,c(1:10)]
pcs <- as.data.frame(pcs)
colnames(pcs) <- paste0(rep("PC",10),rep(1:10))

# add sample ID
pcs$SampleID=rownames(pcs)

covariate_table <- pcs

# Save to file
write.table(covariate_table,
            file = "cohort_10pcs_forregenie.txt",
            quote = FALSE,
            row.names = FALSE,
            col.names = TRUE,
            sep = "\t")
